<a href="https://colab.research.google.com/github/YanaySoker/Quantization/blob/main/qunt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness.git
%cd lm-evaluation-harness

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 8101, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 8101 (delta 46), reused 52 (delta 35), pack-reused 8021
Receiving objects: 100% (8101/8101), 9.48 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (5237/5237), done.
/content/lm-evaluation-harness


In [ ]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/lm-evaluation-harness (from -r requirements.txt (line 1))
     \ 16.5 MB 97 kB/s
     |████████████████████████████████| 441 kB 29.2 MB/s 
     |████████████████████████████████| 5.5 MB 66.4 MB/s 
     |████████████████████████████████| 96 kB 6.5 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 10.1 MB 58.5 MB/s 
     |████████████████████████████████| 191 kB 57.2 MB/s 
     |████████████████████████████████| 2.2 MB 50.4 MB/s 
     |████████████████████████████████| 159 kB 64.3 MB/s 
     |████████████████████████████████| 21.5 MB 1.7 MB/s 
     |████████████████████████████████| 352 kB 58.9 MB/s 
     |████████████████████████████████| 1.3 MB 49.4 MB/s 
     |████████████████████████████████| 45 kB 1.5 MB/s 
     |████████████████████████████████| 4.4 MB 61.3 MB/s 
     |████████████████████████████████| 163 k

In [ ]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install git+https://github.com/alpa-projects/alpa.git as alpa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/alpa-projects/alpa.git to /tmp/pip-req-build-ztn16dd6
  Running command git clone -q https://github.com/alpa-projects/alpa.git /tmp/pip-req-build-ztn16dd6
  Running command git submodule update --init --recursive -q
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [ ]:
! pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile ./lm_eval/evaluator.py
import collections
import itertools
import numpy as np
import random
import lm_eval.metrics
import lm_eval.models
import lm_eval.tasks
import lm_eval.base
from lm_eval.utils import positional_deprecated, run_task_tests


@positional_deprecated
def simple_evaluate(
    model,
    model_args=None,
    tasks=[],
    num_fewshot=0,
    batch_size=None,
    device=None,
    no_cache=False,
    limit=None,
    bootstrap_iters=100000,
    description_dict=None,
    check_integrity=False,
    decontamination_ngrams_path=None,
):

    """Instantiate and evaluate a model on a list of tasks.
    :param model: Union[str, LM]
        Name of model or LM object, see lm_eval.models.get_model
    :param model_args: Optional[str]
        String arguments for each model class, see LM.create_from_arg_string.
        Ignored if `model` argument is a LM object.
    :param tasks: list[Union[str, Task]]
        List of task names or Task objects. Task objects will be taken to have name task.EVAL_HARNESS_NAME if defined and type(task).__name__ otherwise.
    :param num_fewshot: int
        Number of examples in few-shot context
    :param batch_size: int, optional
        Batch size for model
    :param device: str, optional
        PyTorch device (e.g. "cpu" or "cuda:0") for running models
    :param no_cache: bool
        Whether or not to cache
    :param limit: int, optional
        Limit the number of examples per task (only use this for testing)
    :param bootstrap_iters:
        Number of iterations for bootstrap statistics
    :param description_dict: dict[str, str]
        Dictionary of custom task descriptions of the form: `task_name: description`
    :param check_integrity: bool
        Whether to run the relevant part of the test suite for the tasks
    :return
        Dictionary of results
    """
    random.seed(1234)
    np.random.seed(1234)

    assert tasks != [], "No tasks specified"

    if isinstance(model, str):
        if model_args is None:
            model_args = ""
        lm = lm_eval.models.get_model(model).create_from_arg_string(
            model_args, {"batch_size": batch_size, "device": device}
        )
    else:
        assert isinstance(model, lm_eval.base.LM)
        lm = model

    if not no_cache:
        lm = lm_eval.base.CachingLM(
            lm,
            "lm_cache/"
            + model
            + "_"
            + model_args.replace("=", "-").replace(",", "_").replace("/", "-")
            + ".db",
        )

    task_dict = lm_eval.tasks.get_task_dict(tasks)

    if check_integrity:
        run_task_tests(task_list=tasks)

    results = evaluate(
        lm=lm,
        task_dict=task_dict,
        num_fewshot=num_fewshot,
        limit=limit,
        bootstrap_iters=bootstrap_iters,
        description_dict=description_dict,
        decontamination_ngrams_path=decontamination_ngrams_path,
    )

    # add info about the model and few shot config
    results["config"] = {
        "model": model,
        "model_args": model_args,
        "num_fewshot": num_fewshot,
        "batch_size": batch_size,
        "device": device,
        "no_cache": no_cache,
        "limit": limit,
        "bootstrap_iters": bootstrap_iters,
        "description_dict": description_dict,
    }

    return results


decontaminate_suffix = "_decontaminate"


@positional_deprecated
def evaluate(
    lm,
    task_dict,
    provide_description=None,
    num_fewshot=0,
    limit=None,
    bootstrap_iters=100000,
    description_dict=None,
    decontamination_ngrams_path=None,
):
    """Instantiate and evaluate a model on a list of tasks.
    :param lm: obj
        Language Model
    :param task_dict: dict[str, Task]
        Dictionary of tasks. Tasks will be taken to have name task.EVAL_HARNESS_NAME if defined and type(task).__name__ otherwise.
    :param provide_description: bool
        Not implemented, and this option is deprecated and will be removed in a future version in favor of a different description providing method
    :param num_fewshot: int
        Number of examples in few-shot context
    :param limit: int, optional
        Limit the number of examples per task (only use this for testing)
    :param bootstrap_iters:
        Number of iterations for bootstrap statistics
    :param description_dict: dict[str, str]
        Dictionary of custom task descriptions of the form: `task_name: description`
    :return
        Dictionary of results
    """
    # TODO: completely refactor this entire function to not be a huge mess, ideally breaking it down into smaller pieces

    # TODO: todo: implement proper description-providing system
    assert not provide_description  # not implemented.
    if provide_description is not None:
        # nudge people to not specify it at all
        print(
            "WARNING: provide_description is deprecated and will be removed in a future version in favor of description_dict"
        )


    decontaminate = decontamination_ngrams_path is not None

    task_dict_items = [
        (name, task)
        for name, task in task_dict.items()
        if (task.has_validation_docs() or task.has_test_docs())
    ]

    results = collections.defaultdict(dict)
    versions = collections.defaultdict(dict)

    requests = collections.defaultdict(list)
    requests_origin = collections.defaultdict(list)

    overlaps = collections.defaultdict(list)  # {task_name: contaminated_docs}

    # If we ever run into issues where the eval tasks don't fit in memory and we can't afford a machine with bigger
    # memory, we can always modify this plumbing to support that, but I didn't want to include it just yet because
    # over-engineering is bad (or we could make it write the requests to disk and then read them back out again
    #  - probably using an sqlite db because of all the moving parts we have

    # TODO: we need unit tests & sanity checks or something to ensure that the return of `validation_docs` is stable
    docs = {}

    docs_for_decontamination = collections.defaultdict(list)

    # get lists of each type of request
    for task_name, task in task_dict_items:
        versions[task_name] = task.VERSION
        # default to test doc, fall back to val doc if validation unavailable
        # TODO: the test-fallback-to-val system isn't final, we should revisit it at some point
        if task.has_test_docs():
            task_doc_func = task.test_docs
            task_set = "test"  # Required for caching in the decontamination
        elif task.has_validation_docs():
            task_set = "val"  # Required for caching in the decontamination
            task_doc_func = task.validation_docs
        else:
            raise RuntimeError("Task has neither test_docs nor validation_docs")

        # deterministically shuffle docs and chop off the first `limit` because sometimes docs are in some kind of order
        task_docs = list(task_doc_func())
        rnd = random.Random()
        rnd.seed(42)
        rnd.shuffle(task_docs)

        description = (
            description_dict[task_name]
            if description_dict and task_name in description_dict
            else ""
        )
        

        for doc_id, doc in enumerate(itertools.islice(task_docs, 0, limit)):

            if decontaminate and task.should_decontaminate():
                docs_for_decontamination[(task_name, task_set)].append(
                    task.doc_to_decontamination_query(doc)
                )

            docs[(task_name, doc_id)] = doc
            ctx = task.fewshot_context(
                doc=doc, num_fewshot=num_fewshot, rnd=rnd, description=description
            )
            reqs = task.construct_requests(doc, ctx)
            if not isinstance(reqs, (list, tuple)):
                reqs = [reqs]
            for i, req in enumerate(reqs):
                requests[req.request_type].append(req)
                # i: index in requests for a single task instance
                # doc_id: unique id that we can get back to a doc using `docs`
                requests_origin[req.request_type].append((i, task_name, doc, doc_id))

    # Compare all tasks/sets at once to ensure a single training set scan
    if decontaminate:
        from lm_eval.decontamination.decontaminate import get_train_overlap

        print("Finding train/test overlap, please wait...")
        overlaps = get_train_overlap(
            docs_for_decontamination, decontamination_ngrams_path, limit
        )

    # all responses for each (task, doc)
    process_res_queue = collections.defaultdict(list)

    # execute each type of request
    for reqtype, reqs in requests.items():
        # TODO: right now, this code runs multiple separate LM requests for multiple Requests differing
        #       only in index. We could implement some kind of caching, but that would be more of a band-aid
        #       solution. we could also implement some kind of auto-grouping here;
        #       they should end up next to each other.

        print("Running", reqtype, "requests")
        resps = getattr(lm, reqtype)([req.args for req in reqs])
        resps = [
            x if req.index is None else x[req.index] for x, req in zip(resps, reqs)
        ]
        for resp, (i, task_name, doc, doc_id) in zip(resps, requests_origin[reqtype]):
            process_res_queue[(task_name, doc_id)].append((i, resp))
    vals = collections.defaultdict(list)
    
    # unpack results and sort back in order and return control to Task
    for (task_name, doc_id), requests in process_res_queue.items():
        requests.sort(key=lambda x: x[0])
        requests = [x[1] for x in requests]

        task = task_dict[task_name]
        doc = docs[(task_name, doc_id)]

        metrics = task.process_results(doc, requests)
        for metric, value in metrics.items():
            vals[(task_name, metric)].append(value)

            # Re-use the evaluation for the decontaminated set by just ignoring the overlaps
            if decontaminate and task_name in overlaps:
                if doc_id not in overlaps[task_name]:
                    vals[(task_name, metric + decontaminate_suffix)].append(value)
    

    # aggregate results
    for (task_name, metric), items in vals.items():
        task = task_dict[task_name]
        real_metric = metric  # key when looking up the metric with task.aggregation
        if metric.endswith(decontaminate_suffix):
            real_metric = metric.replace(
                decontaminate_suffix, ""
            )  # decontaminated still uses the same metric
        results[task_name][metric] = task.aggregation()[real_metric](items)

        # hotfix: bleu, chrf, ter seem to be really expensive to bootstrap
        # so we run them less iterations. still looking for a cleaner way to do this

        stderr = lm_eval.metrics.stderr_for_metric(
            metric=task.aggregation()[real_metric],
            bootstrap_iters=min(bootstrap_iters, 1000)
            if metric in ["bleu", "chrf", "ter"]
            else bootstrap_iters,
        )

        if stderr is not None:
            results[task_name][metric + "_stderr"] = stderr(items)

    return {"results": dict(results), "versions": dict(versions)}


def make_table(result_dict):
    """Generate table of results."""
    from pytablewriter import MarkdownTableWriter, LatexTableWriter

    md_writer = MarkdownTableWriter()
    latex_writer = LatexTableWriter()
    md_writer.headers = ["Task", "Version", "Metric", "Value", "", "Stderr"]
    latex_writer.headers = ["Task", "Version", "Metric", "Value", "", "Stderr"]

    values = []

    for k, dic in result_dict["results"].items():
        version = result_dict["versions"][k]
        for m, v in dic.items():
            if m.endswith("_stderr"):
                continue

            if m + "_stderr" in dic:
                se = dic[m + "_stderr"]
                values.append([k, version, m, "%.4f" % v, "±", "%.4f" % se])
            else:
                values.append([k, version, m, "%.4f" % v, "", ""])
            k = ""
            version = ""
    md_writer.value_matrix = values
    latex_writer.value_matrix = values

    # todo: make latex table look good
    # print(latex_writer.dumps())

    return md_writer.dumps()

Overwriting ./lm_eval/evaluator.py


In [ ]:
!rm ./lm_eval/models/qu.py

rm: cannot remove './lm_eval/models/qu.py': No such file or directory


In [ ]:
%%writefile ./lm_eval/models/qu.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.autograd.function import InplaceFunction
import time
import copy
from transformers.pytorch_utils import Conv1D

#from transformers import models
#from transformers.models import gpt2
from transformers.models.gpt2.configuration_gpt2 import GPT2Config

from transformers.models.gpt2.modeling_gpt2 import GPT2Block, GPT2MLP, GPT2Attention
from typing import *

from lm_eval import utils
from . import gpt2
from . import gpt3
from . import dummy


class new_conv1d(Conv1D):
    def __init__(self, cnv, B, E_bits, m):
        super().__init__(1,1)

        self.E_bits = E_bits
        self.B = int(B)
        self.m = m

        self.training = copy.deepcopy(cnv.training)
        self._parameters = copy.deepcopy(cnv._parameters)
        self._buffers = copy.deepcopy(cnv._buffers)
        self._non_persistent_buffers_set = copy.deepcopy(cnv._non_persistent_buffers_set)
        self._backward_hooks = copy.deepcopy(cnv._backward_hooks)
        self._is_full_backward_hook = copy.deepcopy(cnv._is_full_backward_hook)
        self._forward_hooks = copy.deepcopy(cnv._forward_hooks)
        self._forward_pre_hooks = copy.deepcopy(cnv._forward_pre_hooks)
        self._state_dict_hooks = copy.deepcopy(cnv._state_dict_hooks)
        self._load_state_dict_pre_hooks = copy.deepcopy(cnv._load_state_dict_pre_hooks)
        self._load_state_dict_post_hooks = copy.deepcopy(cnv._load_state_dict_post_hooks)
        self._modules = copy.deepcopy(cnv._modules)

        self.nf = copy.deepcopy(cnv.nf)
        t = copy.deepcopy(cnv.weight)
        self.weight = torch.nn.Parameter(qunt(t, self.B, self.E_bits, self.m))

        s = copy.deepcopy(cnv.bias)
        self.bias = torch.nn.Parameter(qunt(s, self.B, self.E_bits, self.m))
    

    def forward(self, x):
        x = qunt(x, self.B, self.E_bits, self.m)

        size_out = x.size()[:-1] + (self.nf,)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(size_out)
        return x


class new_embd(nn.Embedding):
    def __init__(self, embd, B, E_bits, m):
        super().__init__(embd.num_embeddings, embd.embedding_dim)

        self.E_bits = E_bits
        self.B = int(B)
        self.m = m

        self.num_embeddings = copy.deepcopy(embd.num_embeddings)
        self.embedding_dim = copy.deepcopy(embd.embedding_dim)
        self.padding_idx = copy.deepcopy(embd.padding_idx)
        self.max_norm = copy.deepcopy(embd.max_norm)
        self.norm_type = copy.deepcopy(embd.norm_type)
        self.scale_grad_by_freq = copy.deepcopy(embd.scale_grad_by_freq)
        t = copy.deepcopy(embd.weight)
        self.weight = torch.nn.Parameter(qunt(t, self.B, self.E_bits, self.m))

        self.sparse = copy.deepcopy(embd.sparse)


        self.training = copy.deepcopy(embd.training)
        self._parameters = copy.deepcopy(embd._parameters)
        self._buffers = copy.deepcopy(embd._buffers)
        self._non_persistent_buffers_set = copy.deepcopy(embd._non_persistent_buffers_set)
        self._backward_hooks = copy.deepcopy(embd._backward_hooks)
        self._is_full_backward_hook = copy.deepcopy(embd._is_full_backward_hook)
        self._forward_hooks = copy.deepcopy(embd._forward_hooks)
        self._forward_pre_hooks = copy.deepcopy(embd._forward_pre_hooks)
        self._state_dict_hooks = copy.deepcopy(embd._state_dict_hooks)
        self._load_state_dict_pre_hooks = copy.deepcopy(embd._load_state_dict_pre_hooks)
        self._load_state_dict_post_hooks = copy.deepcopy(embd._load_state_dict_post_hooks)
        self._modules = copy.deepcopy(embd._modules)
    

    def forward(self, input: torch._tensor.Tensor) -> torch._tensor.Tensor:
        input = qunt(input, self.B, self.E_bits, self.m)
        return F.embedding(
            input, self.weight, self.padding_idx, self.max_norm,
            self.norm_type, self.scale_grad_by_freq, self.sparse)


class new_gpt2mlp(GPT2MLP):
    def __init__(self, mlp, B, E_bits, m):
        config = GPT2Config()
        inner_dim = config.n_inner if config.n_inner is not None else 4 * config.hidden_size
        super().__init__(inner_dim, config)

        self.E_bits = E_bits
        self.B = int(B)
        self.m = m

        self.training = copy.deepcopy(mlp.training)
        self._parameters = copy.deepcopy(mlp._parameters)
        self._buffers = copy.deepcopy(mlp._buffers)
        self._non_persistent_buffers_set = copy.deepcopy(mlp._non_persistent_buffers_set)
        self._backward_hooks = copy.deepcopy(mlp._backward_hooks)
        self._is_full_backward_hook = copy.deepcopy(mlp._is_full_backward_hook)
        self._forward_hooks = copy.deepcopy(mlp._forward_hooks)
        self._forward_pre_hooks = copy.deepcopy(mlp._forward_pre_hooks)
        self._state_dict_hooks = copy.deepcopy(mlp._state_dict_hooks)
        self._load_state_dict_pre_hooks = copy.deepcopy(mlp._load_state_dict_pre_hooks)
        self._load_state_dict_post_hooks = copy.deepcopy(mlp._load_state_dict_post_hooks)
        self._modules = copy.deepcopy(mlp._modules)

        self.c_fc = new_conv1d(mlp.c_fc, self.B, self.E_bits, self.m)
        self.c_proj = new_conv1d(mlp.c_proj, self.B, self.E_bits, self.m)
        self.act = copy.deepcopy(mlp.act)
        self.dropout = copy.deepcopy(mlp.dropout)

    
    def forward(self, hidden_states: Optional[Tuple[torch.FloatTensor]]) -> torch.FloatTensor:
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        hidden_states = self.c_fc(hidden_states)
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        hidden_states = self.act(hidden_states)
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        hidden_states = self.c_proj(hidden_states)
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)
        
        hidden_states = self.dropout(hidden_states)
        return hidden_states


class new_GPT2Attention(GPT2Attention):
    def __init__(self, attn, B, E_bits, m):
        super().__init__(GPT2Config())

        self.E_bits = E_bits
        self.B = int(B)
        self.m = m

        self.training = copy.deepcopy(attn.training)
        self._parameters = copy.deepcopy(attn._parameters)
        self._buffers = copy.deepcopy(attn._buffers)
        self._non_persistent_buffers_set = copy.deepcopy(attn._non_persistent_buffers_set)
        self._backward_hooks = copy.deepcopy(attn._backward_hooks)
        self._is_full_backward_hook = copy.deepcopy(attn._is_full_backward_hook)
        self._forward_hooks = copy.deepcopy(attn._forward_hooks)
        self._forward_pre_hooks = copy.deepcopy(attn._forward_pre_hooks)
        self._state_dict_hooks = copy.deepcopy(attn._state_dict_hooks)
        self._load_state_dict_pre_hooks = copy.deepcopy(attn._load_state_dict_pre_hooks)
        self._load_state_dict_post_hooks = copy.deepcopy(attn._load_state_dict_post_hooks)
        self._modules = copy.deepcopy(attn._modules)

        self.embed_dim = copy.deepcopy(attn.embed_dim)
        self.num_heads = copy.deepcopy(attn.num_heads)
        self.head_dim = copy.deepcopy(attn.head_dim)
        self.split_size = copy.deepcopy(attn.split_size)
        self.scale_attn_weights = copy.deepcopy(attn.scale_attn_weights)
        self.is_cross_attention = copy.deepcopy(attn.is_cross_attention)

        self.scale_attn_by_inverse_layer_idx = copy.deepcopy(attn.scale_attn_by_inverse_layer_idx)
        self.layer_idx = copy.deepcopy(attn.layer_idx)
        self.reorder_and_upcast_attn = copy.deepcopy(attn.reorder_and_upcast_attn)

        if self.is_cross_attention:
            self.c_attn = new_conv1d(copy.deepcopy(attn.c_attn), self.B, self.E_bits, self.m)
            self.q_attn = new_conv1d(copy.deepcopy(attn.q_attn), self.B, self.E_bits, self.m)
        else:
            self.c_attn = new_conv1d(copy.deepcopy(attn.c_attn), self.B, self.E_bits, self.m)
        self.c_proj = new_conv1d(copy.deepcopy(attn.c_proj), self.B, self.E_bits, self.m)

        self.attn_dropout = copy.deepcopy(attn.attn_dropout)
        self.resid_dropout = copy.deepcopy(attn.resid_dropout)
        self.pruned_heads = copy.deepcopy(attn.pruned_heads)

    
    def _attn(self, query, key, value, attention_mask=None, head_mask=None):
        attn_weights = torch.matmul(query, key.transpose(-1, -2))

        attn_weights = qunt(attn_weights, self.B, self.E_bits, self.m)

        if self.scale_attn_weights:
            attn_weights = attn_weights / torch.tensor(
                value.size(-1) ** 0.5, dtype=attn_weights.dtype, device=attn_weights.device
            )

        # Layer-wise attention scaling
        if self.scale_attn_by_inverse_layer_idx:
            attn_weights = attn_weights / float(self.layer_idx + 1)
        
        attn_weights = qunt(attn_weights, self.B, self.E_bits, self.m)

        if not self.is_cross_attention:
            # if only "normal" attention layer implements causal mask
            query_length, key_length = query.size(-2), key.size(-2)
            causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length].to(torch.bool)
            mask_value = torch.finfo(attn_weights.dtype).min
            # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
            # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
            mask_value = torch.tensor(mask_value, dtype=attn_weights.dtype).to(attn_weights.device)
            attn_weights = torch.where(causal_mask, attn_weights, mask_value)

        if attention_mask is not None:
            # Apply the attention mask
            attn_weights = attn_weights + attention_mask

        attn_weights = nn.functional.softmax(attn_weights, dim=-1)

        attn_weights = qunt(attn_weights, self.B, self.E_bits, self.m)

        # Downcast (if necessary) back to V's dtype (if in mixed-precision) -- No-Op otherwise
        attn_weights = attn_weights.type(value.dtype)
        attn_weights = self.attn_dropout(attn_weights)

        # Mask heads if we want to
        if head_mask is not None:
            attn_weights = attn_weights * head_mask

        attn_output = torch.matmul(attn_weights, value)

        return attn_output, attn_weights

    
    def _upcast_and_reordered_attn(self, query, key, value, attention_mask=None, head_mask=None):
        # Use `torch.baddbmm` (a bit more efficient w/ alpha param for scaling -- from Megatron-LM)
        bsz, num_heads, q_seq_len, dk = query.size()
        _, _, k_seq_len, _ = key.size()

        # Preallocate attn_weights for `baddbmm`
        attn_weights = torch.empty(bsz * num_heads, q_seq_len, k_seq_len, dtype=torch.float32, device=query.device)

        # Compute Scale Factor
        scale_factor = 1.0
        if self.scale_attn_weights:
            scale_factor /= float(value.size(-1)) ** 0.5

        if self.scale_attn_by_inverse_layer_idx:
            scale_factor /= float(self.layer_idx + 1)

        # Upcast (turn off autocast) and reorder (Scale K by 1 / root(dk))
        with autocast(enabled=False):
            q, k = query.reshape(-1, q_seq_len, dk), key.transpose(-1, -2).reshape(-1, dk, k_seq_len)
            attn_weights = torch.baddbmm(attn_weights, q.float(), k.float(), beta=0, alpha=scale_factor)
            attn_weights = attn_weights.reshape(bsz, num_heads, q_seq_len, k_seq_len)

            attn_weights = qunt(attn_weights, self.B, self.E_bits, self.m)

        if not self.is_cross_attention:
            # if only "normal" attention layer implements causal mask
            query_length, key_length = query.size(-2), key.size(-2)
            causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length].bool()
            mask_value = torch.finfo(attn_weights.dtype).min
            # Need to be a tensor, otherwise we get error: `RuntimeError: expected scalar type float but found double`.
            # Need to be on the same device, otherwise `RuntimeError: ..., x and y to be on the same device`
            mask_value = torch.tensor(mask_value, dtype=attn_weights.dtype).to(attn_weights.device)
            attn_weights = torch.where(causal_mask, attn_weights, mask_value)

        if attention_mask is not None:
            # Apply the attention mask
            attn_weights = attn_weights + attention_mask

        attn_weights = nn.functional.softmax(attn_weights, dim=-1)

        attn_weights = qunt(attn_weights, self.B, self.E_bits, self.m)

        # Downcast (if necessary) back to V's dtype (if in mixed-precision) -- No-Op if otherwise
        if attn_weights.dtype != torch.float32:
            raise RuntimeError("Error with upcasting, attn_weights does not have dtype torch.float32")
        attn_weights = attn_weights.type(value.dtype)
        attn_weights = self.attn_dropout(attn_weights)

        # Mask heads if we want to
        if head_mask is not None:
            attn_weights = attn_weights * head_mask

        attn_output = torch.matmul(attn_weights, value)

        attn_output = qunt(attn_output, self.B, self.E_bits, self.m)

        return attn_output, attn_weights

    
    def forward(
        self,
        hidden_states: Optional[Tuple[torch.FloatTensor]],
        layer_past: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = False,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[Union[torch.Tensor, Tuple[torch.Tensor]], ...]:

        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        if encoder_hidden_states is not None:
            if not hasattr(self, "q_attn"):
                raise ValueError(
                    "If class is used as cross attention, the weights `q_attn` have to be defined. "
                    "Please make sure to instantiate class with `GPT2Attention(..., is_cross_attention=True)`."
                )

            query = self.q_attn(hidden_states)
            key, value = self.c_attn(encoder_hidden_states).split(self.split_size, dim=2)
            attention_mask = encoder_attention_mask
        else:
            query, key, value = self.c_attn(hidden_states).split(self.split_size, dim=2)

        query = self._split_heads(query, self.num_heads, self.head_dim)
        key = self._split_heads(key, self.num_heads, self.head_dim)
        value = self._split_heads(value, self.num_heads, self.head_dim)

        if layer_past is not None:
            past_key, past_value = layer_past
            key = torch.cat((past_key, key), dim=-2)
            value = torch.cat((past_value, value), dim=-2)

        query = qunt(query, self.B, self.E_bits, self.m)
        key = qunt(key, self.B, self.E_bits, self.m)
        value = qunt(value, self.B, self.E_bits, self.m)

        if use_cache is True:
            present = (key, value)
        else:
            present = None

        if self.reorder_and_upcast_attn:
            attn_output, attn_weights = self._upcast_and_reordered_attn(query, key, value, attention_mask, head_mask)
        else:
            attn_output, attn_weights = self._attn(query, key, value, attention_mask, head_mask)

        attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)
        attn_output = self.c_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        outputs = (attn_output, present)
        if output_attentions:
            outputs += (attn_weights,)

        return outputs  # a, present, (attentions)


class new_GPT2Block(GPT2Block):
    def __init__(self, blck, B, E_bits, m):
        super().__init__(GPT2Config())

        self.training = copy.deepcopy(blck.training)
        self._parameters = copy.deepcopy(blck._parameters)
        self._buffers = copy.deepcopy(blck._buffers)
        self._non_persistent_buffers_set = copy.deepcopy(blck._non_persistent_buffers_set)
        self._backward_hooks = copy.deepcopy(blck._backward_hooks)
        self._is_full_backward_hook = copy.deepcopy(blck._is_full_backward_hook)
        self._forward_hooks = copy.deepcopy(blck._forward_hooks)
        self._forward_pre_hooks = copy.deepcopy(blck._forward_pre_hooks)
        self._state_dict_hooks = copy.deepcopy(blck._state_dict_hooks)
        self._load_state_dict_pre_hooks = copy.deepcopy(blck._load_state_dict_pre_hooks)
        self._load_state_dict_post_hooks = copy.deepcopy(blck._load_state_dict_post_hooks)
        self._modules = copy.deepcopy(blck._modules)

        self.E_bits = E_bits
        self.B = int(B)
        self.m = m

        self.ln_1 = copy.deepcopy(blck.ln_1)
        self.attn = new_GPT2Attention(blck.attn, self.B, self.E_bits, self.m)
        
        self.ln_2 = Module()
        self.ln_2 = copy.deepcopy(blck.ln_2)

        try:
            self.crossattention = new_GPT2Attention(blck.crossattention, self.B, self.E_bits, self.m)
            self.ln_cross_attn = copy.deepcopy(blck.ln_cross_attn)
        except:
            pass

        self.mlp = new_gpt2mlp(blck.mlp, self.B, self.E_bits, self.m)


    def forward(
        self,
        hidden_states: Optional[Tuple[torch.FloatTensor]],
        layer_past: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = False,
        output_attentions: Optional[bool] = False,
    ) -> Union[Tuple[torch.Tensor], Optional[Tuple[torch.Tensor, Tuple[torch.FloatTensor, ...]]]]:
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        residual = hidden_states
        hidden_states = self.ln_1(hidden_states)

        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        attn_outputs = self.attn(
            hidden_states,
            layer_past=layer_past,
            attention_mask=attention_mask,
            head_mask=head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
        )
        attn_output = attn_outputs[0]  # output_attn: a, present, (attentions)

        outputs = attn_outputs[1:]
        # residual connection
        hidden_states = attn_output + residual
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        if encoder_hidden_states is not None:
            # add one self-attention block for cross-attention
            if not hasattr(self, "crossattention"):
                raise ValueError(
                    f"If `encoder_hidden_states` are passed, {self} has to be instantiated with "
                    "cross-attention layers by setting `config.add_cross_attention=True`"
                )
            residual = hidden_states
            hidden_states = self.ln_cross_attn(hidden_states)

            cross_attn_outputs = self.crossattention(
                hidden_states,
                attention_mask=attention_mask,
                head_mask=head_mask,
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_attention_mask,
                output_attentions=output_attentions,
            )
            attn_output = cross_attn_outputs[0]

            # residual connection
            hidden_states = residual + attn_output

            outputs = outputs + cross_attn_outputs[2:]  # add cross attentions if we output attention weights

        residual = hidden_states
        hidden_states = self.ln_2(hidden_states)
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        feed_forward_hidden_states = self.mlp(hidden_states)
        # residual connection
        hidden_states = residual + feed_forward_hidden_states
        hidden_states = qunt(hidden_states, self.B, self.E_bits, self.m)

        if use_cache:
            outputs = (hidden_states,) + outputs
        else:
            outputs = (hidden_states,) + outputs[1:]

        return outputs  # hidden_states, present, (attentions, cross_attentions)


class qu(gpt2.GPT2LM):
    def __init__(self, B, *args, **kwargs):
        super(qu, self).__init__(*args, **kwargs)

        self.E_bits = 2
        self.B = int(B)
        self.m = 5

        self.E = 2**self.E_bits

        self.quantizeFwd = True

        # update wte
        self.gpt2.transformer._modules["old_wte"]=copy.deepcopy(self.gpt2.transformer._modules["wte"])
        self.gpt2.transformer._modules["wte"] = new_embd(self.gpt2.transformer.old_wte, self.B, self.E_bits, self.m)
       
        # update wpe
        self.gpt2.transformer._modules["old_wpe"]=copy.deepcopy(self.gpt2.transformer._modules["wpe"])
        self.gpt2.transformer._modules["wpe"] = new_embd(self.gpt2.transformer.old_wpe, self.B, self.E_bits, self.m)

        # updae h
        for i in range(len(self.gpt2.transformer.h)):
            mo = self.gpt2.transformer.h[i]
            new_m = new_GPT2Block(mo, self.B, self.E_bits, self.m)
            self.gpt2.transformer.h[i] = new_m
    

    def new_wte(self, input):
          input = qunt(input, self.B, self.E_bits, self.m)
          return self.gpt2.transformer.old_wte(input)
        

def qunt(input, B, E_bits, m):
    isint = (input.type()==input.int().type())
    islong = (input.type()==input.long().type())

    # To positive
    sign = (input.__ge__(0).int()-0.5)*2
    abs_input = input * sign

    # Upper clipping
    Max = (2 - 2 ** (-m)) * (2 ** (2 ** E_bits - B))
    abs_input = torch.clip(abs_input, max=Max)

    # Lower clipping (just for exponent classes), for avoiding log(0) and log(<=0.5)
    abs_input_without_0 = torch.clip(abs_input, min=2 ** (1-B))

    # Exponent classes
    exp_range = torch.log2(abs_input_without_0)
    exp_range = exp_range+B
    exp_range = exp_range.int()
    exp_range = exp_range.float()
    exp_range = exp_range-B

    # 1 if is not in the smaller exponent-class (union of 0 and 1)
    min_mask = torch.ge(exp_range, 1.5 - B).int()

    # Maximum value to include in this exponent class
    upper_bound = 2 ** (exp_range + 1)

    # Minimum value to include in this exponent class
    lower_bound = 2 ** exp_range
    lower_bound = lower_bound * min_mask

    # Number of values in class
    num_of_value = 2**m
    num_of_values = (2 - min_mask) * num_of_value

    abs_input = abs_input - lower_bound
    abs_input = abs_input * num_of_values
    abs_input = abs_input / (upper_bound - lower_bound)


    abs_input = abs_input + 0.5
    abs_input = abs_input.int()
    abs_input = abs_input * (upper_bound - lower_bound)
    abs_input = abs_input / num_of_values
    abs_input = abs_input + lower_bound
    
    x = abs_input * sign
    if isint:
      x = x.int()
    if islong:
      x = x.long()

    return x

Overwriting ./lm_eval/models/qu.py


In [ ]:
!rm ./lm_eval/models/__init__.py

In [ ]:
%%writefile ./lm_eval/models/__init__.py
from . import gpt2
from . import gpt3
from . import dummy

from . import qu

MODEL_REGISTRY = {
    "qu": qu.qu,
    "gpt2": gpt2.GPT2LM,
}


def get_model(model_name):
    return MODEL_REGISTRY[model_name]

Writing ./lm_eval/models/__init__.py


In [ ]:
!rm ./main.py

In [ ]:
%%writefile ./main.py
import argparse
import json
import logging
import fnmatch
import matplotlib.pyplot as plt


from lm_eval import tasks, evaluator

logging.getLogger("openai").setLevel(logging.WARNING)


class MultiChoice:
    def __init__(self, choices):
        self.choices = choices

    # Simple wildcard support (linux filename patterns)
    def __contains__(self, values):
        for value in values.split(","):
            if len(fnmatch.filter(self.choices, value)) == 0:
                return False

        return True

    def __iter__(self):
        for choice in self.choices:
            yield choice


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", required=True)
    parser.add_argument("--model_args", default="")
    parser.add_argument("--tasks", default=None, choices=MultiChoice(tasks.ALL_TASKS))
    parser.add_argument("--provide_description", action="store_true")
    parser.add_argument("--num_fewshot", type=int, default=0)
    parser.add_argument("--batch_size", type=int, default=None)
    parser.add_argument("--device", type=str, default=None)
    parser.add_argument("--output_path", default=None)
    parser.add_argument("--limit", type=int, default=None)
    parser.add_argument("--no_cache", action="store_true")
    parser.add_argument("--decontamination_ngrams_path", default=None)
    parser.add_argument("--description_dict_path", default=None)
    parser.add_argument("--check_integrity", action="store_true")

    return parser.parse_args()


# Returns a list containing all values of the source_list that
# match at least one of the patterns
def pattern_match(patterns, source_list):
    task_names = set()
    for pattern in patterns:
        for matching in fnmatch.filter(source_list, pattern):
            task_names.add(matching)
    return list(task_names)


def main():
    args = parse_args()

    assert not args.provide_description  # not implemented

    if args.limit:
        print(
            "WARNING: --limit SHOULD ONLY BE USED FOR TESTING. REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT."
        )

    if args.tasks is None:
        task_names = tasks.ALL_TASKS
    else:
        task_names = pattern_match(args.tasks.split(","), tasks.ALL_TASKS)

    print(f"Selected Tasks: {task_names}")

    description_dict = {}
    if args.description_dict_path:
        with open(args.description_dict_path, "r") as f:
            description_dict = json.load(f)

    R = dict()

    _from, _to = -1, 2

    for i in range(_from, _to):
        print(f"B = {i}:")
        results = evaluator.simple_evaluate(
            model=args.model,
            model_args=f"B={i}",
            tasks=task_names,
            num_fewshot=args.num_fewshot,
            batch_size=args.batch_size,
            device=args.device,
            no_cache=args.no_cache,
            limit=args.limit,
            description_dict=description_dict,
            decontamination_ngrams_path=args.decontamination_ngrams_path,
            check_integrity=args.check_integrity,
        )
        results = results['results']
        for K in results.keys():
            for k in results[K].keys():
                if "_stderr" not in k:
                    if f"{K}: {k}" not in R.keys():
                        R[f"{K}: {k}"] = []
                    R[f"{K}: {k}"].append(results[K][k])
                    print(f"{K}: {k}: {results[K][k]}")

    _keys = list(R.keys())
    for k in _keys:
        print(f"{k}: {R[k]}")
        plt.plot(range(_from, _to), R[k], label=k)
    plt.legend(_keys)
    plt.show()

if __name__ == "__main__":
    main()

Writing ./main.py


In [ ]:
!python main.py \
	--model qu \
	--device 0 \
	--tasks hellaswag

Selected Tasks: ['hellaswag']
B = -1:
Using device '0'
#####
100% 3/3 [00:00<00:00, 752.03it/s]
Running loglikelihood requests
100% 40145/40145 [1:31:29<00:00,  7.31it/s]
G2
G5
G6
hellaswag: acc: 0.26020713005377416
hellaswag: acc_norm: 0.2544313881696873
B = 0:
Using device '0'
#####
100% 3/3 [00:00<00:00, 692.66it/s]
Running loglikelihood requests
  0% 173/40145 [00:24<1:35:49,  6.95it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x7fea77c00cd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1210, in __iter__
    self.close()
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1289, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 595, in _decr_instances
    cls._instances.remove(instance)
  File "/usr/lib/python3.7/_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
  File "/usr/local/lib/python3.7/dist-packages/tqdm/utils.py", l